In [ ]:
import numpy as np
import os
from matplotlib import image
import matplotlib.pyplot as plt
import cv2 as cv

from terrain_nerf.autonav import arc

# autoreload
%load_ext autoreload
%autoreload 2

Img to costmap

In [ ]:
plt.figure(figsize=(7,7))
img = image.imread('../data/airsim/images/birdseye_1687738527.7947364.png')
plt.imshow(img)

In [ ]:
# Convert to grayscale
gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# Downsample to 41x41
gray = cv.resize(gray, (41,41))
# Invert
gray = 1.0 - gray
# Automatically set values in center to 0
gray[20-2:20+2, 20-2:20+2] = np.min(gray)
plt.figure(figsize=(7,7))
plt.imshow(gray, cmap='viridis_r')
# Show colorbar
plt.colorbar()

System ID arcs

In [ ]:
data = np.load("../data/airsim/systemID/arcs_blocks_dynamic.npz")
arcs = data['arcs']
steer_rates = data['steer_rates']
# close file
data.close()

In [ ]:
# Plot arcs
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
for i, a1 in enumerate(arcs):
    ax.plot(a1[:, 0] - 8.75, a1[:, 1], label=f"steer_rate={steer_rates[i]}", c='k')
for w in steer_rates:
    a2 = arc(np.zeros(3), [2.5, 1.6*w], 50, 0.1)
    ax.plot(a2[:, 0], a2[:, 1], label=f"steer_rate={w}", c='r')
ax.set_xlabel("x")
ax.set_ylabel("y")
#ax.legend()
plt.axis('equal')
# Invert y axis
plt.gca().invert_yaxis()
plt.show()

In [ ]:
steer_rates

In [ ]:
a2[-1]

In [ ]:
a1[-1] - np.array([8.75, 0])

In [ ]:
from terrain_nerf.autonav import local_to_global

pose = np.array([1, 1, -np.pi/2])
arc_global = local_to_global(pose, a2)
# Plot local vs global
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
ax.plot(a2[:, 0], a2[:, 1], label="local", c='k')
ax.plot(arc_global[:, 0], arc_global[:, 1], label="global", c='r')
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.legend()
plt.axis('equal')
# Invert y axis
plt.gca().invert_yaxis()
plt.show()

In [ ]:
# Plot arcs
fig, ax = plt.subplots(1, 1, figsize=(7, 7))
for w in np.linspace(-0.25, 0.25, 11):
    a = arc(np.zeros(3), [2.5, w], 50, 0.1)
    ax.plot(a[:, 0], a[:, 1], label=f"steer_rate={w}")
plt.axis('equal')
plt.show()

In [ ]:
from terrain_nerf.autonav import AutoNav

autonav = AutoNav(np.array([0, 0]))
autonav.update_costmap(None)

In [ ]:
autonav.replan()

In [ ]:
# Display costmap
plt.imshow(autonav.costmap)

In [ ]:
cand_arcs = autonav.candidate_arcs
opt_arc, opt_cost, opt_w = autonav.get_next_arc()

# Plot candidate arcs and optimal arc
plt.figure(figsize=(10,10))
for arc in cand_arcs:
    plt.plot(arc[:,0], arc[:,1], 'b')
plt.plot(opt_arc[:,0], opt_arc[:,1], 'r')
plt.imshow(autonav.costmap)